In [9]:
import glob
import pandas as pd
from sklearn import preprocessing

In [10]:
def merge_data_frames():
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    #combine all files in the list
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    #export to csv
    combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')
    

In [11]:
# merge_data_frames()
df=pd.read_csv("combined_csv.csv",index_col=0)
print(df.shape)
copy=df.drop_duplicates(keep='first',ignore_index=False).copy()
print(copy.shape)

(15792, 57)
(15166, 57)


In [12]:
def fill_empty_values(df):
    df.fillna("Empty", inplace=True)
fill_empty_values(copy)


In [13]:
# Because there are few types of details in each item I want to separate the types to diffrent columns. 
#There are many diffrent details types so I took only over 100 freaqwency types.

def divide_details_col():
    dictionaroro= {}
    sorted_dict={}
    list1=[]
    for i in copy['Details']:
        for j in i.split(', '):
            if(j not in dictionaroro.keys()):
                dictionaroro[j]=1 
            else:
                dictionaroro[j]=dictionaroro[j]+1
    sorted_keys = sorted(dictionaroro, key=dictionaroro.get,reverse=True)

    for w in sorted_keys:
        if(dictionaroro[w]>=100 and w!='Empty'):
            sorted_dict[w] = dictionaroro[w]
            list1.append(w)
            
    for x in list1:
        copy[x]=(copy['Details'].str.contains(x)).astype(int)

divide_details_col()

In [14]:
def remove_empty_cols(col_list):   
    copy.drop(axis=1, index=None, columns=col_list, inplace=True)

col_list=['Details','Composition','Mouw lengte','Materiaal','Bra Type','Chest pad','Panty Type','Item NamePrice','Doorzichtig','Arabian Clothing','Care Instructions','Kleur','Lengte','Neklijn','Patroon type','Seizoen','Stijl','Type mouwen','Stof','Type fit']
remove_empty_cols(col_list)

In [18]:
# filterd all the damaged items (the items with incorrect number reviews)  
sum_cols=(copy['1 Star Reviews'] + copy['2 Stars Reviews'] + copy['3 Stars Reviews']+copy['4 Stars Reviews'] + copy['5 Stars Reviews'])
copy=copy[~((sum_cols>copy['Reviews Num']) & ((copy['Reviews Num']==copy['1 Star Reviews']) | (copy['Reviews Num']==copy['2 Stars Reviews']) | (copy['Reviews Num']==copy['3 Stars Reviews']) | (copy['Reviews Num']==copy['4 Stars Reviews']) | (copy['Reviews Num']==copy['5 Stars Reviews'])))]

In [19]:
# filterd all items with low reviews(less then 50)
copy=copy[copy["Reviews Num"] > 50]

In [20]:
#save results table 
copy.to_csv( "combined_with_classification_2.csv", index=False, encoding='utf-8-sig')